In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-8xsoojzo
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-8xsoojzo
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=378229e145a106bb2408e758f8ea51ac9049578db73a1b165aae60a1d2325ced
  Stored in directory: /tmp/pip-ephem-wheel-cache-haxqf1zm/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [8]:
%%cuda --name LDF.cu

#include "/content/drive/MyDrive/graph/coloring.h"
#include "/content/drive/MyDrive/graph/graph_d.h"

#define THREADxBLOCK 128

using namespace std;

__global__ void findIS (Coloring* col, GraphStruct *str, uint* weigths, bool* candidateNodes) {
	uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

	uint offset = str->cumDegs[idx];
	uint deg = str->cumDegs[idx + 1] - str->cumDegs[idx];

	bool candidate = true;
  for (uint j = 0; j < deg; j++) {
		uint neighID = str->neighs[offset + j];
    uint neighDeg = str->cumDegs[neighID + 1] - str->cumDegs[neighID];
		if (col->coloring[neighID] == 0 &&
				((deg < neighDeg) ||
				((deg == neighDeg) && (weigths[idx] < weigths[neighID])))) {
			candidate = false;
		}
	}


 if(candidate){
		candidateNodes[idx] = true;
  }

}

__global__ void colorer (Coloring* col, GraphStruct *str, bool* candidateNodes) {
	uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

	uint offset = str->cumDegs[idx];
	uint deg = str->cumDegs[idx + 1] - str->cumDegs[idx];

  if(candidateNodes[idx]){
      bool* forbidden;
			uint n = str->nodeSize;
			cudaMalloc((void**) &forbidden, n * sizeof(bool));
			memset(forbidden, false, n);

			for (uint j = 0; j < deg; j++) {
					uint neighID = str->neighs[offset + j];
					forbidden[col->coloring[neighID]] = true;
			}

			for(uint i = 1; i <= n; i++){
				if(forbidden[i] == false){
					col->coloring[idx] = i;
					free(forbidden);
					return;
				}
			}
		free(forbidden);
	}

	else
	 		col->uncoloredNodes = true;

}

Coloring* graphColoring(GraphStruct *str) {
  Coloring* col;
	CHECK(cudaMallocManaged(&col, sizeof(Coloring)));
	uint n = str->nodeSize;
	col->uncoloredNodes = true;

  CHECK(cudaMallocManaged( &(col->coloring), n * sizeof(uint)));
	memset(col->coloring,0,n);

	uint* weigths;
  curandState_t* states;
	cudaMalloc((void**) &states, n * sizeof(curandState_t));
	cudaMalloc((void**) &weigths, n * sizeof(uint));
	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );
	uint seed = 0;

	init <<< blocks, threads >>> (seed, states, weigths, n);

	bool* candidateNodes;
	cudaMalloc((void**) &candidateNodes, n * sizeof(bool));
	cudaMemset(candidateNodes, false, n);



  col->numOfColors = 0;
	while (col->uncoloredNodes) {
		col->uncoloredNodes = false;
    col->numOfColors++;
		findIS <<< blocks, threads >>> (col, str, weigths, candidateNodes);
		colorer <<< blocks, threads >>> (col, str, candidateNodes);
		cudaDeviceSynchronize();
	}

	cudaFree(states);
	return col;
}

__global__ void init (uint seed, curandState_t* states, uint* weigths, uint n) {
		uint idx  =threadIdx.x + blockDim.x * blockIdx.x;

		if(idx > n){
			return;
		}

		weigths[idx] = idx;
}


'File written in /content/src/LDF.cu'

In [6]:
%%cuda --name test_LDF.cu

#include "/content/drive/MyDrive/graph/coloring.h"
#include "/content/drive/MyDrive/graph/graph_d.h"


int main(void) {
	unsigned int n = 5;		 // number of nodes for random graphs
	float prob = 0.5;				    // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed

	srand(time(0));
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

	// new graph with n nodes
	Graph graph(n,1);

	// generate a random graph
	graph.randGraph(prob,eng);

	// get the graph struct
	GraphStruct *str = graph.getStruct();

  cudaEventRecord(start);

	print_d<<<1,1>>>(str, true);

	Coloring* col = graphColoring(str);
	cudaDeviceSynchronize();

	cudaEventRecord(stop);
  cudaEventSynchronize(stop);

	//Stampo in millisecondi quanto tempo ci ha messo a colorare il grafo.
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("%f ms\n", milliseconds);

	//printColoring(col, str, 1);
 printf("Coloratura trovata: ");
	for(int i = 0; i < str->nodeSize; i++){
			printf("%d ", col->coloring[i]);
	}

	return EXIT_SUCCESS;
}

'File written in /content/src/test_LDF.cu'

In [9]:
!nvcc -dc src/test_LDF.cu /content/src/LDF.cu /content/drive/MyDrive/graph/graph.cpp /content/drive/MyDrive/graph/graph_d.cu
!nvcc test_LDF.o LDF.o graph.o graph_d.o -o test_LDF
!./test_LDF

** Graph (num node: 5, num edges: 6)
  node(0)[3]-> 1 2 3 
  node(1)[2]-> 0 4 
  node(2)[2]-> 0 4 
  node(3)[2]-> 0 4 
  node(4)[3]-> 1 2 3 

0 0 0 0 0 1 1 1 1 1 2 2 2 2 2 3 3 3 3 3 4 4 4 4 4 




0 0 0 1 1 1 2 2 2 3 3 3 4 4 4 


2.863136 ms
Coloratura trovata: 1 2 2 2 1 